In [95]:
from sklearn.model_selection import cross_val_score, train_test_split, cross_validate
from fairlearn.datasets import fetch_adult
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

from concurrent.futures import ThreadPoolExecutor
import tqdm as notebook_tqdm
from metrics import (
    equality_opportunity_difference,
    predictive_equality_difference,
    predictive_parity_difference,
    metrics,
    average_absolute_odds_difference,
    metric_evaluation, 
    get_metric_evaluation,
    
)
from fairlearn.metrics import demographic_parity_difference
from sklearn.utils import resample

import numpy as np
import pandas as pd
import optuna
import dill
import pickle

from sklearn.metrics import (
    f1_score, 
    confusion_matrix, 
    make_scorer, 
    accuracy_score, 
    recall_score, 
    matthews_corrcoef,
    precision_score
)

In [96]:
numeric_transformer = Pipeline(
    steps=[
        ("impute", SimpleImputer()),
        ("scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore")),
    ]
)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)


n_sim = 0
sensitive_col = 'sex'
file_name = 'results/sex/f1-ppv-models-motpe-succesivehalving-parallel-150trials-4sim.pkl'

with open(file_name, 'rb') as in_strm:
    results = dill.load(in_strm)

sensitive_attribute = 'sex'
sim_n = 1
data = fetch_adult(as_frame=True)
X_raw = data.data
y = (data.target == ">50K") * 1

if sensitive_attribute == 'race':
    mapping = {'White':'white','Black':'black','Asian-Pac-Islander':'others','Amer-Indian-Eskimo':'others','Other':'others'}
    X_raw.loc[:,'race'] = X_raw['race'].map(mapping).astype("category")

perc = .5
X_raw, y = resample(X_raw, y, n_samples=int(perc*X_raw.shape[0]), random_state = 123)  
  
(X_train, X_test, y_train, y_test) = train_test_split(
    X_raw, y, test_size=0.8, stratify=y, random_state=sim_n
)

data_dict = {}
data_dict['X_train'] = X_train.reset_index(drop=True)
data_dict['X_test'] = X_test.reset_index(drop=True)
data_dict['y_train'] = y_train.reset_index(drop=True)
data_dict['y_test'] = y_test.reset_index(drop=True)

study = results[0]
metrics = get_metrics(study, data_dict, sensitive_col, preprocessor)
metrics = get_default_metrics(metrics, data_dict, sensitive_col, preprocessor)
metrics['file_name'] = file_name

In [98]:
def detailed_objective(trial, data_dict, sensitive_col, preprocessor):
    classifier_name = trial.suggest_categorical("classifier", ["RF", 'GBM','LGBM'])

    if classifier_name == "logit":        
        params = {
            "penalty" : trial.suggest_categorical('logit_penalty', ['l1','l2']),
            "C" : trial.suggest_float('logit_c', 0.001, 10),
            "max_iter": 2000,
            "solver" : 'saga'
            }
        classifier = LogisticRegression(**params)

    elif classifier_name =="RF":
        params = {
            'n_estimators': trial.suggest_int("rf_n_estimators", 100, 1000),
            'criterion': trial.suggest_categorical("rf_criterion", ['gini', 'entropy']),
            'max_depth': trial.suggest_int("rf_max_depth", 1, 4),
            'min_samples_split': trial.suggest_float("rf_min_samples_split", 0.01, 1),
            }
        classifier = RandomForestClassifier(**params)

    elif classifier_name =="LGBM":
        params = {
            'n_estimators': trial.suggest_int("lgbm_n_estimators", 20, 10000),
            'num_leaves': trial.suggest_int("lgbm_num_leaves", 10, 1000),
            'max_depth': trial.suggest_int("lgbm_max_depth", 2, 20),
            'min_child_samples': trial.suggest_int("lgbm_min_child_samples", 5, 300),
            'learning_rate': trial.suggest_float('lgbm_learning_rate', 1e-5, 1e-2),
            'boosting_type': trial.suggest_categorical("lgbm_boosting_type", ['goss', 'gbdt'])
            }
        classifier = LGBMClassifier(**params)  

    elif classifier_name =="GBM":
        params = {
            'n_estimators': trial.suggest_int("gbm_n_estimators", 100, 1000), 
            'criterion': trial.suggest_categorical("gbm_criterion", ['squared_error', 'friedman_mse']),
            'max_depth': trial.suggest_int("gbm_max_depth", 1, 4),
            'min_samples_split': trial.suggest_int("gbm_min_samples_split", 5, 300),
            }
        classifier = GradientBoostingClassifier(**params)            

    else:
        None

    pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", classifier),
        ]
    )

    pipeline.fit(data_dict['X_train'], data_dict['y_train'])
    y_pred = pipeline.predict(data_dict['X_test'])
    metrics = metric_evaluation(
        y_true= data_dict['y_test'], 
        y_pred= y_pred, 
        sensitive_features=data_dict['X_test'][sensitive_col]
        )
    return classifier_name, metrics


def get_default_metrics(metrics, data_dict, sensitive_col, preprocessor):
    models = metrics['overall']['model_name'].unique()
    classifier = {
        'logit' : LogisticRegression(),
        'GBM' : GradientBoostingClassifier(),
        'LGBM' : LGBMClassifier(),
        'RF' : RandomForestClassifier(),
    }

    metrics['default_overall'] = pd.DataFrame()
    metrics['default_bygroup'] = pd.DataFrame()
    for model in models:
        clf = classifier[model]
        pipeline = Pipeline(
            steps=[
                ("preprocessor", preprocessor),
                ("classifier", clf),
            ]
        )

        pipeline.fit(data_dict['X_train'], data_dict['y_train'])
        y_pred = pipeline.predict(data_dict['X_test'])
        metric_frame = metric_evaluation(
            y_true= data_dict['y_test'], 
            y_pred=y_pred, 
            sensitive_features=data_dict['X_test'][sensitive_col]
        )
        # Overall
        fair_records = pd.DataFrame.from_records([get_metric_evaluation(metric_frame)])
        new_metric_overall = pd.concat([fair_records, pd.DataFrame(metric_frame.overall).T], axis = 1)
        new_metric_overall['model'] = model
        metrics['default_overall'] = pd.concat([metrics['default_overall'], new_metric_overall])
        # By group
        new_metric_bygroup = metric_frame.by_group.reset_index()
        new_metric_bygroup['model'] = model
        metrics['default_bygroup'] = pd.concat([metrics['default_bygroup'], new_metric_bygroup])
    return metrics

def get_metrics(study, data_dict, sensitive_col, preprocessor):
    metrics = {}
    metrics['overall'] = pd.DataFrame()
    metrics['bygroup'] = pd.DataFrame()
    #metrics['fair_metric'] = study.user_attrs['fair_metric']
    #metrics['model_metric'] = study.user_attrs['model_metric']
    i = 1
    for best_trial in study.best_trials:
        if best_trial.values != [0,0]:
            fair_value, model_value = best_trial.values
            clf_name, metric = detailed_objective(best_trial, data_dict, sensitive_col, preprocessor)
            # Overall
            fair_records = pd.DataFrame.from_records([get_metric_evaluation(metric)])
            new_metric_overall = pd.concat([fair_records, pd.DataFrame(metric.overall).T], axis = 1)
            new_metric_overall['best_trial'] = i
            new_metric_overall['fair_metric'] = fair_value
            new_metric_overall['model_metric'] = model_value
            new_metric_overall['model_name'] = clf_name
            metrics['overall'] = pd.concat([metrics['overall'], new_metric_overall])
            # By Groups
            new_metric_bygroup = metric.by_group.reset_index()
            new_metric_bygroup['best_trial'] = i
            metrics['bygroup'] = pd.concat([metrics['bygroup'], new_metric_bygroup])
            i += 1
    return metrics

In [99]:

n_sim = 0
sensitive_col = 'sex'
file_name = 'results/sex/f1-ppv-models-motpe-succesivehalving-parallel-150trials-4sim.pkl'

with open(file_name, 'rb') as in_strm:
    results = dill.load(in_strm)

study = results[0]
metrics = get_metrics(study, data_dict, sensitive_col, preprocessor)
metrics = get_default_metrics(metrics, data_dict, sensitive_col, preprocessor)
metrics['file_name'] = file_name